# Example 5: pyVDERM lite
For full functionality, pyVDERM requires pymeshlab, which can cause conflicts in some environments. However, pymeshlab is only used for converting mesh files to point clouds, and later converting point clouds back to meshes. A lite version of pyVDERM is available with no pymeshlab dependency.

In [9]:
import pyVDERM as vd
import numpy as np

## Import point cloud file
Using read_xyz, read in a point cloud directly, skipping the mesh step. This point cloud would typically be created using the full pyVDERM, pymeshlab, open3D, or some other similar package, in a different environment or machine that has no conflicts.

The read_xyz function expects a row to look like one of the following
1. ```x y z u_x u_y u_z rho```
2. ```x y z u_x u_y u_z ```
3. ```x y z rho```
4. ```x y z```

Where u is some vector associated with the point (most often either a vector normal to the surface, or the velocity of the point at a given time) and rho is the denity of the point. As long as the components are ordered as position, vector, density, the function can read them even if some are missing. All 3 will always be returned, but if they are missing they will be returned as None

In [10]:
points, norms, dens = vd.read_xyz('cube.xyz')
# here we have saved points and normal vectors, but no densities
print(points.shape,norms.shape,dens)

(14797, 3) (14797, 3) None


## Set up grid and assign densities
We can now proceed as usual in setting up the deformation

In [11]:
grid_params = vd.make_initial_grid(points,max_points=32_000)
vd.print_grid_info(grid_params)

GRID INFORMATION

Grid dimensions: 32 × 32 × 32 = 32,768 points
Grid spacing (h): 0.355476

Grid size:   [11.0198, 11.0198, 11.0198]
Object size: [5.5099, 5.5099, 5.5099]
Ratio:       [2.00x, 2.00x, 2.00x]

Grid bounds:
  x: [-5.5099, 5.5099]
  y: [-5.5099, 5.5099]
  z: [-5.5099, 5.5099]

Object bounds:
  x: [-2.7549, 2.7549]
  y: [-2.7549, 2.7549]
  z: [-2.7549, 2.7549]

Object margins (distance from grid boundary):
  x: min=2.7549, max=2.7549
  y: min=2.7549, max=2.7549
  z: min=2.7549, max=2.7549

Verification for padding ratios:
  x: 2.000x
  y: 2.000x
  z: 2.000x


In [12]:
def periodic_density(x,y,z):
    # a density function can reference the grid parameters
    L = grid_params['shape'][0]
    return 10 + 9.99*np.sin(8*32/L*np.pi*x/(L-1))*np.cos(4*32/L*np.pi*y/(L-1))*np.cos(4*32/L*np.pi*z/(L-1))

vdgrid = vd.VDERMGrid(shape=grid_params['shape'],h=grid_params['h'],min_bounds=grid_params['min_bounds'])
vdgrid.set_density(periodic_density)

## Run the deformation
run_VDERM will run normally without pymeshlab. run_VDERM_with_tracking will run normally if asked to export grids and/or surfaces, but will throw an import error if asked to export meshes with pyVDERM lite. All visualization functions except export_meshes_to_paraview and export_all_to_paraview will work normally.

In [13]:
deformed_grid = vd.run_VDERM(vdgrid, n_max=500)

Converged:  62%|██████▏   | 308/500 [01:18<00:48,  3.93it/s, ε=1.993e-02, target=2.000e-02]


Converged at iteration 308


## Create and export the deformed object point cloud
Because pymeshlab's point cloud functions can infer normal vectors, we export only the points and the densities. If normal vectors are needed for remeshing, the original normal vectors from the non-deformed object usually work well enough for a remeshing function.

In [8]:
deformed_surface = vd.interpolate_to_surface(points, grid_params, deformed_grid.get_displacement_field())
interpolated_densities = vd.interpolate_densities(points,vdgrid)
vd.write_xyz('deformed_cube.xyz',deformed_surface,densities=interpolated_densities)